Pruebas PSN

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import datetime
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup as bs
import requests



In [2]:
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
# options.add_argument('--start-maximized') SOLO EN PC SOBREMESA si fuera necesario.

Prueba primera con google

In [3]:
driver = webdriver.Chrome(service=service, options=options)

driver.get("http://www.google.es")

time.sleep(3)

rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/span/div/div/div/div[3]/div[1]/button[1]/div')
rechazar_cookies.click()

time.sleep(3)

driver.quit()

Haciendo pruebas con psn

In [4]:
df_juegos = pd.DataFrame(columns=["Titulo","Precio_original","Precio_actual","Precio con mayor rebaja"])

In [5]:
df_juegos

,Titulo,Precio_original,Precio_actual,Precio con mayor rebaja


In [ ]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ###

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://store.playstation.com/")


lista_tiempo = [2.5,3, 3.5, 4,4.5, 5, 6.5, 7]

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

#seleccion de juego

cont = 1
while cont <= 24:

    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    #Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')

    # Variable por juego
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()

    try:
        precio_original = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original = soup.find("span",class_="psw-t-title-m").get_text()
    try:
        precio_actual = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual = precio_original
   
    precio_con_mayor_rebaja = None
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Precio_original":precio_original,"Precio_actual":precio_actual,"Precio con mayor rebaja":precio_con_mayor_rebaja} 
    time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    cont += 1
driver.quit()


1 minuto aprox cada juego.

In [27]:
df_juegos

,Titulo,Precio_original,Precio_actual,Precio con mayor rebaja
0,01 Deadliest Zone Catch — Boat Crab & Fishing ...,"14,99 €","8,99 €",None
1,01 Deadliest Zone Catch — Boat Crab & Fishing ...,"14,99 €","8,99 €",None
2,01 Deadliest Zone Catch — Boat Crab & Fishing ...,"14,99 €","8,99 €",None
3,01 Deadliest Zone Catch — Boat Crab & Fishing ...,"14,99 €","8,99 €",None
4,0 Degrees,"4,99 €","4,99 €",None
5,0 Rally Desert Race: Offroad Dirt Simulator,"14,99 €","8,99 €",None
6,1001 Spikes,"14,99 €","14,99 €",None
7,100ft Robot Golf,"9,99 €","9,99 €",None
8,101 Ways to Die,"9,99 €","9,99 €",None
9,10 Second Ninja X,"9,99 €","9,99 €",None


In [61]:
#Titulo
soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()

'01 Deadliest Zone Catch — Boat Crab & Fishing Simulator'

In [62]:
#Precio Original sin PSN
soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()


'14,99\xa0€'

In [63]:
#Precio Actual sin PSN
soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()

'8,99\xa0€'